In [1]:
import pandas as pd
import numpy as np
import sklearn
import torch
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('./data/all_games.csv')

In [3]:
data.shape

(66080, 67)

In [6]:
data.head()

,Unnamed: 0,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,0,1249669800,Aug 07 2009 - 6:30pm,complete,NaN,Wolfsburg,Stuttgart,Felix Brych,1.0,0.0,0.0,1.35,1.53,2,0,2,0,0,0,"71,82",NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VOLKSWAGEN ARENA (Wolfsburg)
1,1,1249738200,Aug 08 2009 - 1:30pm,complete,NaN,Borussia Dortmund,Köln,Wolfgang Stark,1.0,0.0,0.0,2.00,1.35,1,0,1,0,0,0,75,NaN,-1,-1,0,0,1,0,0,0,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Signal-Iduna-Park (Dortmund)
2,2,1249738200,Aug 08 2009 - 1:30pm,complete,NaN,Nürnberg,Schalke 04,Knut Kircher,1.0,0.0,0.0,1.12,1.76,1,2,3,1,0,1,88,"36,50",-1,-1,2,0,1,0,1,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Max-Morlock-Stadion (Nürnberg)
3,3,1249738200,Aug 08 2009 - 1:30pm,complete,NaN,Werder Bremen,Eintracht Frankfurt,NaN,1.0,0.0,0.0,1.76,1.18,2,3,5,4,2,2,"13,44","6,42,71",-1,-1,2,1,2,0,1,2,0,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,wohninvest WESERSTADION (Bremen)
4,4,1249738200,Aug 08 2009 - 1:30pm,complete,NaN,Hertha BSC,Hannover 96,Guido Winkmann,1.0,0.0,0.0,0.53,0.82,1,0,1,0,0,0,82,NaN,-1,-1,3,0,2,0,0,3,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Olympiastadion Berlin (Berlin)


In [ ]:
"""
metoda clean data

do dropnięcia: Unnamed: 0, timestamp, status, home_team_goal_timings, away_team_goal_timings
do dropnięcia mecze anulowane
zmienna o tym czy mecz był rozgrywany na stadionie gospodarza czy nie
"""

In [7]:
def clean_data(data):
    #transforming date variable
    data['date_GMT'] = pd.to_datetime(data['date_GMT'])
    data['date'] = pd.to_datetime(data['date_GMT'].dt.date)
    data['time'] = data['date_GMT'].dt.time
    
    #adding aggregated variables
    data['corners_total'] = data['home_team_corner_count'] + data['away_team_corner_count']
    data['yellow_cards_total'] = data['home_team_yellow_cards'] + data['away_team_yellow_cards']
    data['red_cards_total'] = data['home_team_red_cards'] + data['away_team_red_cards']
    data['cards_total'] = data['yellow_cards_total'] + data['red_cards_total']
    data['shots_total'] = data['home_team_shots'] + data['away_team_shots']
    data['shots_on_target_total'] = data['home_team_shots_on_target'] + data['away_team_shots_on_target']
    data['shots_off_target_total'] = data['home_team_shots_off_target'] + data['away_team_shots_off_target']
    data['fouls_total'] = data['home_team_fouls'] + data['away_team_fouls']
    
    #getting stadium names without city in brackets
    data['base_name'] = data['stadium_name'].str.replace(r" \(.*\)$", "", regex=True)

    city_map = data[data['stadium_name'].str.contains(r"\(.*\)")].copy()
    city_map['city'] = city_map['stadium_name'].str.extract(r"\((.*?)\)")[0]
    city_map = city_map.groupby('base_name')['city'].agg(pd.Series.mode).to_dict()

    #filling stadium names without city in brackets
    data['normalized_stadium'] = data.apply(lambda row: f"{row['base_name']} ({city_map.get(row['base_name'], 'Unknown')})" if '(' not in row['stadium_name'] else row['stadium_name'], axis=1)

    #dropping unnecessary features
    data.drop(['Unnamed: 0', 'timestamp', 'status', 'home_team_goal_timings', 'away_team_goal_timings', 'date_GMT'], axis=1, inplace=True)
    
    #setting one of the target variable - result of the game
    data['result'] = np.where(data['home_team_goal_count'] == data['away_team_goal_count'], 0, np.where(data['home_team_goal_count'] > data['away_team_goal_count'], 1, 2))
    
    return data

In [8]:
data = clean_data(data)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_5028\2951951484.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['date_GMT'] = pd.to_datetime(data['date_GMT'])


ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [9]:
data.isna().sum()

Unnamed: 0                                 0
timestamp                                  0
date_GMT                                   0
status                                     0
attendance                             21359
home_team_name                             0
away_team_name                             0
referee                                24472
Game Week                               8325
Pre-Match PPG (Home)                       0
Pre-Match PPG (Away)                       0
home_ppg                                   0
away_ppg                                   0
home_team_goal_count                       0
away_team_goal_count                       0
total_goal_count                           0
total_goals_at_half_time                   0
home_team_goal_count_half_time             0
away_team_goal_count_half_time             0
home_team_goal_timings                 15583
away_team_goal_timings                 21901
home_team_corner_count                     0
away_team_

In [17]:
data[data['stadium_name'].isna()]['home_team_name'].value_counts(normalize=True)

home_team_name
Burnley                    0.025025
Reading                    0.024024
Sheffield United           0.024024
Birmingham City            0.023023
Wolverhampton Wanderers    0.023023
                             ...   
Vllaznia Shkodër           0.001001
Gomel                      0.001001
Struga                     0.001001
Wolfsberger AC             0.001001
St. Johnstone              0.001001
Name: proportion, Length: 177, dtype: float64

In [19]:
data[data['stadium_name'].isna()]['home_team_name'].unique()

array(['Birmingham City', 'Queens Park Rangers', 'Plymouth Argyle',
       'Ipswich Town', 'Derby County', 'Crystal Palace', 'Coventry City',
       'Charlton Athletic', 'Cardiff City', 'Blackpool',
       'Sheffield Wednesday', 'Nottingham Forest',
       'Wolverhampton Wanderers', 'Watford', 'Swansea City',
       'Southampton', 'Sheffield United', 'Reading', 'Preston North End',
       'Norwich City', 'Doncaster Rovers', 'Barnsley', 'Burnley',
       'Bristol City', 'Opava', 'Příbram', 'Zlín', 'Teplice',
       'Sigma Olomouc', 'Karviná', 'Haarlem', 'MVV', 'Fortuna Sittard',
       'Cambuur', 'RBC', 'Telstar', 'Oss', 'Go Ahead Eagles',
       'Almere City', 'De Graafschap', 'Volendam', 'Excelsior',
       'ADO Den Haag', 'Willem II', 'PEC Zwolle', 'PSV', 'Utrecht',
       'Sparta Rotterdam', 'Ajax', 'RKC Waalwijk', 'VVV', 'Heracles',
       'Twente', 'Vitesse', 'Heerenveen', 'Feyenoord', 'Emmen',
       'Groningen', 'AZ', 'Sileks', 'Rīgas FS', 'Sūduva', 'Honka',
       'Dundalk', 'S

In [23]:
# Wyodrębnienie nazw stadionów bez miasta
data['base_name'] = data['stadium_name'].str.replace(r" \(.*\)$", "", regex=True)

# Tworzenie mapy nazw z najczęściej występującym miastem w nawiasie dla każdego stadionu
city_map = data[data['stadium_name'].str.contains(r"\(.*\)")].copy()
city_map['city'] = city_map['stadium_name'].str.extract(r"\((.*?)\)")[0]
city_map = city_map.groupby('base_name')['city'].agg(pd.Series.mode).to_dict()

# Uzupełnianie miast dla nazw stadionów bez nawiasów
data['normalized_stadium'] = data.apply(lambda row: f"{row['base_name']} ({city_map.get(row['base_name'], 'Unknown')})" if '(' not in row['stadium_name'] else row['stadium_name'], axis=1)

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [22]:
for team in data[data['stadium_name'].isna()]['home_team_name'].unique():
    print(data[data['home_team_name'] == team]['stadium_name'].value_counts(normalize=True, dropna=False))
    print('-----')

stadium_name
St Andrew's Trillion Trophy Stadium (Birmingham)    0.518006
St. Andrew's Stadium                                0.310249
St Andrew's Stadium (Birmingham)                    0.108033
NaN                                                 0.063712
Name: proportion, dtype: float64
-----
stadium_name
Kiyan Prince Foundation Stadium (London)    0.479042
Loftus Road Stadium                         0.383234
NaN                                         0.068862
Loftus Road (London)                        0.068862
Name: proportion, dtype: float64
-----
stadium_name
NaN                     0.5
Home Park (Plymouth)    0.5
Name: proportion, dtype: float64
-----
stadium_name
Portman Road (Ipswich- Suffolk)    0.460630
Portman Road Stadium               0.448819
NaN                                0.090551
Name: proportion, dtype: float64
-----
stadium_name
Pride Park Stadium (Derby)    0.598266
iPro Stadium (Derby)          0.315029
NaN                           0.066474
Pride Park Stadium

In [11]:
data['status'].value_counts(normalize=True)

status
complete    0.99655
canceled    0.00345
Name: proportion, dtype: float64

In [12]:
data['stadium_name'].value_counts()

stadium_name
Stadio Giuseppe Meazza (Milano)                  420
Stadio Olimpico (Roma)                           410
The City Ground (Nottingham- Nottinghamshire)    321
St. James' Park (Newcastle upon Tyne)            319
Stadio Comunale Luigi Ferraris (Genova)          311
                                                ... 
Batumi Arena (Adjarabet Arena) (Batumi)            1
Við Djúpumýrar (Klaksvík- Borðoy)                  1
Yeni Sakarya Atatürk Stadyumu (Adapazarı)          1
DG Arena (Podgorica)                               1
Park Avenue (Aberystwyth- Ceredigion)              1
Name: count, Length: 1499, dtype: int64

In [15]:
data[data['home_team_name'] == 'Rangers']['stadium_name'].value_counts(dropna=False)

stadium_name
Ibrox Stadium (Glasgow)    112
Ibrox Stadium               63
NaN                          3
Name: count, dtype: int64

In [16]:
data['home_team_name'].nunique()

892

In [ ]:
data[(data['home_team_name'] == 'Rangers') & (data['stadium_name'] == )]

# Daty

In [24]:
data['date_GMT'] = pd.to_datetime(data['date_GMT'])
data['date'] = pd.to_datetime(data['date_GMT'].dt.date)
data['time'] = data['date_GMT'].dt.time

In [25]:
data.drop('date_GMT', axis=1, inplace=True)

In [26]:
data.head()

,Unnamed: 0,timestamp,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,base_name
0,0,1249669800,complete,NaN,Wolfsburg,Stuttgart,Felix Brych,1.0,0.0,0.0,1.35,1.53,2,0,2,0,0,0,"71,82",NaN,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,VOLKSWAGEN ARENA (Wolfsburg),2009-08-07,18:30:00,-2,-2,-2,-4,-2,-2,-2,-2,VOLKSWAGEN ARENA
1,1,1249738200,complete,NaN,Borussia Dortmund,Köln,Wolfgang Stark,1.0,0.0,0.0,2.00,1.35,1,0,1,0,0,0,75,NaN,-1,-1,0,0,1,0,0,0,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Signal-Iduna-Park (Dortmund),2009-08-08,13:30:00,-2,1,0,1,-2,-2,-2,-2,Signal-Iduna-Park
2,2,1249738200,complete,NaN,Nürnberg,Schalke 04,Knut Kircher,1.0,0.0,0.0,1.12,1.76,1,2,3,1,0,1,88,"36,50",-1,-1,2,0,1,0,1,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Max-Morlock-Stadion (Nürnberg),2009-08-08,13:30:00,-2,3,0,3,-2,-2,-2,-2,Max-Morlock-Stadion
3,3,1249738200,complete,NaN,Werder Bremen,Eintracht Frankfurt,NaN,1.0,0.0,0.0,1.76,1.18,2,3,5,4,2,2,"13,44","6,42,71",-1,-1,2,1,2,0,1,2,0,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,wohninvest WESERSTADION (Bremen),2009-08-08,13:30:00,-2,4,1,5,-2,-2,-2,-2,wohninvest WESERSTADION
4,4,1249738200,complete,NaN,Hertha BSC,Hannover 96,Guido Winkmann,1.0,0.0,0.0,0.53,0.82,1,0,1,0,0,0,82,NaN,-1,-1,3,0,2,0,0,3,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Olympiastadion Berlin (Berlin),2009-08-08,13:30:00,-2,5,0,5,-2,-2,-2,-2,Olympiastadion Berlin


In [27]:
data.dtypes

Unnamed: 0                                      int64
timestamp                                       int64
status                                         object
attendance                                    float64
home_team_name                                 object
away_team_name                                 object
referee                                        object
Game Week                                     float64
Pre-Match PPG (Home)                          float64
Pre-Match PPG (Away)                          float64
home_ppg                                      float64
away_ppg                                      float64
home_team_goal_count                            int64
away_team_goal_count                            int64
total_goal_count                                int64
total_goals_at_half_time                        int64
home_team_goal_count_half_time                  int64
away_team_goal_count_half_time                  int64
home_team_goal_timings      

## Zmienna - sezon

In [28]:
def assign_season(date):
    year = date.year
    if date.month >= 8:  # Sezon zaczyna się w sierpniu
        return f'{str(year)[2:]}/{str(year+1)[2:]}'
    else:
        return f'{str(year-1)[2:]}/{str(year)[2:]}'

In [29]:
data['season'] = data['date'].apply(assign_season)

In [31]:
data.sort_values(by='date')

,Unnamed: 0,timestamp,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_goal_timings,away_team_goal_timings,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,base_name,season
48683,48683,1186832700,complete,NaN,Sunderland,Tottenham Hotspur,NaN,NaN,0.00,0.00,1.58,0.89,1,0,1,0,0,0,90'3,NaN,-1,-1,1,0,1,0,0,1,1,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Stadium of Light (Sunderland),2007-08-11,11:45:00,-2,2,0,2,-2,-2,-2,-2,Stadium of Light,07/08
48689,48689,1186848900,complete,NaN,Aston Villa,Liverpool,NaN,NaN,0.00,0.00,1.74,1.79,1,2,3,1,0,1,84,"30,87",-1,-1,4,0,2,0,1,3,0,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Villa Park (Birmingham),2007-08-11,16:15:00,-2,6,0,6,-2,-2,-2,-2,Villa Park,07/08
48688,48688,1186840800,complete,NaN,Bolton Wanderers,Newcastle United,NaN,NaN,0.00,0.00,1.37,0.74,1,3,4,3,0,3,50,"11,21,26",-1,-1,1,0,1,0,0,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,University of Bolton Stadium (Bolton),2007-08-11,14:00:00,-2,2,0,2,-2,-2,-2,-2,University of Bolton Stadium,07/08
48684,48684,1186840800,complete,NaN,West Ham United,Manchester City,NaN,NaN,0.00,0.00,1.47,0.95,0,2,2,1,0,1,NaN,"18,87",-1,-1,0,0,4,0,0,0,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Boleyn Ground (London),2007-08-11,14:00:00,-2,4,0,4,-2,-2,-2,-2,Boleyn Ground,07/08
48686,48686,1186840800,complete,NaN,Everton,Wigan Athletic,NaN,NaN,0.00,0.00,1.95,0.58,2,1,3,1,1,0,"26,75",80,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Goodison Park (Liverpool),2007-08-11,14:00:00,-2,-2,-2,-4,-2,-2,-2,-2,Goodison Park,07/08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66078,66078,1686157200,complete,NaN,Ankaragücü,Adana Demirspor,Çağdaş Altay,38.0,1.29,1.53,1.22,1.61,1,2,3,1,0,1,82,"31,57",7,1,0,0,0,0,0,0,0,0,13,9,8,6,5,3,13,9,51,49,1.65,1.82,1.96,1.40,3.00,68,88,59,32,27,35,71,62,88,10.30,4.35,2.30,3.5,2.50,1.10,1.73,1.95,3.15,1.39,3.11,Eryaman Stadyumu (Ankara),2023-06-07,17:00:00,8,0,0,0,22,

In [32]:
data.isna().sum()

Unnamed: 0                                 0
timestamp                                  0
status                                     0
attendance                             21359
home_team_name                             0
away_team_name                             0
referee                                24472
Game Week                               8325
Pre-Match PPG (Home)                       0
Pre-Match PPG (Away)                       0
home_ppg                                   0
away_ppg                                   0
home_team_goal_count                       0
away_team_goal_count                       0
total_goal_count                           0
total_goals_at_half_time                   0
home_team_goal_count_half_time             0
away_team_goal_count_half_time             0
home_team_goal_timings                 15583
away_team_goal_timings                 21901
home_team_corner_count                     0
away_team_corner_count                     0
home_team_

In [33]:
data_clean = data.dropna(subset=['stadium_name'])

In [35]:
data_clean.shape

(65081, 78)

In [37]:
data.isna().sum()

Unnamed: 0                                 0
timestamp                                  0
status                                     0
attendance                             21359
home_team_name                             0
away_team_name                             0
referee                                24472
Game Week                               8325
Pre-Match PPG (Home)                       0
Pre-Match PPG (Away)                       0
home_ppg                                   0
away_ppg                                   0
home_team_goal_count                       0
away_team_goal_count                       0
total_goal_count                           0
total_goals_at_half_time                   0
home_team_goal_count_half_time             0
away_team_goal_count_half_time             0
home_team_goal_timings                 15583
away_team_goal_timings                 21901
home_team_corner_count                     0
away_team_corner_count                     0
home_team_

In [39]:
data_clean.drop(['Unnamed: 0', 'timestamp', 'status', 'attendance', 'referee', 'Game Week', 'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_team_goal_timings', 'away_team_goal_timings',
                 'stadium_name', 'base_name'], axis=1, inplace=True)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_5028\3156604440.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean.drop(['Unnamed: 0', 'timestamp', 'status', 'attendance', 'referee', 'Game Week', 'Pre-Match PPG (Home)', 'Pre-Match PPG (Away)', 'home_team_goal_timings', 'away_team_goal_timings',


In [42]:
data_clean.dtypes

home_team_name                                 object
away_team_name                                 object
home_ppg                                      float64
away_ppg                                      float64
home_team_goal_count                            int64
away_team_goal_count                            int64
total_goal_count                                int64
total_goals_at_half_time                        int64
home_team_goal_count_half_time                  int64
away_team_goal_count_half_time                  int64
home_team_corner_count                          int64
away_team_corner_count                          int64
home_team_yellow_cards                          int64
home_team_red_cards                             int64
away_team_yellow_cards                          int64
away_team_red_cards                             int64
home_team_first_half_cards                      int64
home_team_second_half_cards                     int64
away_team_first_half_cards  

In [41]:
data_clean['result'] = np.where(data_clean['home_team_goal_count'] == data_clean['away_team_goal_count'], 0, np.where(data_clean['home_team_goal_count'] > data_clean['away_team_goal_count'], 1, 2))

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_5028\2921145499.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['result'] = np.where(data_clean['home_team_goal_count'] == data_clean['away_team_goal_count'], 0, np.where(data_clean['home_team_goal_count'] > data_clean['away_team_goal_count'], 1, 2))


In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FootballModel(nn.Module):
    def __init__(self, num_teams, embedding_dim, num_features):
        super(FootballModel, self).__init__()
        self.team_embeddings = nn.Embedding(num_teams, embedding_dim)
        self.fc1 = nn.Linear(embedding_dim * 2 + num_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)  # Liczba goli, może być zmodyfikowane dla innych wyników

    def forward(self, home_team, away_team, features):
        home_embed = self.team_embeddings(home_team)
        away_embed = self.team_embeddings(away_team)
        
        # Łączenie osadzeń drużyn z dodatkowymi cechami
        x = torch.cat((home_embed, away_embed, features), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [83]:
data_clean

,home_team_name,away_team_name,home_ppg,away_ppg,home_team_goal_count,away_team_goal_count,total_goal_count,total_goals_at_half_time,home_team_goal_count_half_time,away_team_goal_count_half_time,home_team_corner_count,away_team_corner_count,home_team_yellow_cards,home_team_red_cards,away_team_yellow_cards,away_team_red_cards,home_team_first_half_cards,home_team_second_half_cards,away_team_first_half_cards,away_team_second_half_cards,home_team_shots,away_team_shots,home_team_shots_on_target,away_team_shots_on_target,home_team_shots_off_target,away_team_shots_off_target,home_team_fouls,away_team_fouls,home_team_possession,away_team_possession,Home Team Pre-Match xG,Away Team Pre-Match xG,team_a_xg,team_b_xg,average_goals_per_match_pre_match,btts_percentage_pre_match,over_15_percentage_pre_match,over_25_percentage_pre_match,over_35_percentage_pre_match,over_45_percentage_pre_match,over_15_HT_FHG_percentage_pre_match,over_05_HT_FHG_percentage_pre_match,over_15_2HG_percentage_pre_match,over_05_2HG_percentage_pre_match,average_corners_per_match_pre_match,average_cards_per_match_pre_match,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,date,time,corners_total,yellow_cards_total,red_cards_total,cards_total,shots_total,shots_on_target_total,shots_off_target_total,fouls_total,season,result
0,Wolfsburg,Stuttgart,1.35,1.53,2,0,2,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,2009-08-07,18:30:00,-2,-2,-2,-4,-2,-2,-2,-2,09/10,1
1,Borussia Dortmund,Köln,2.00,1.35,1,0,1,0,0,0,-1,-1,0,0,1,0,0,0,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,2009-08-08,13:30:00,-2,1,0,1,-2,-2,-2,-2,09/10,1
2,Nürnberg,Schalke 04,1.12,1.76,1,2,3,1,0,1,-1,-1,2,0,1,0,1,1,0,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,2009-08-08,13:30:00,-2,3,0,3,-2,-2,-2,-2,09/10,2
3,Werder Bremen,Eintracht Frankfurt,1.76,1.18,2,3,5,4,2,2,-1,-1,2,1,2,0,1,2,0,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,2009-08-08,13:30:00,-2,4,1,5,-2,-2,-2,-2,09/10,2
4,Hertha BSC,Hannover 96,0.53,0.82,1,0,1,0,0,0,-1,-1,3,0,2,0,0,3,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,0.00,0.00,0.00,0.00,0.00,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,2009-08-08,13:30:00,-2,5,0,5,-2,-2,-2,-2,09/10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66075,Fenerbahçe,Gazişehir Gaziantep,2.28,0.67,3,0,3,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2.06,1.42,0.00,0.00,3.45,47,85,68,36,27,33,68,47,83,11.07,5.64,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,2023-06-07,13:00:00,-2,-2,-2,-4,-2,-2,-2,-2,22/23,1
66076,Giresunspor,Antalyaspor,1.22,0.78,2,0,2,1,1,0,3,11,2,1,1,0,1,2,1,0,6,17,4,6,2,11,13,12,31,69,1.80,1.29,0.91,2.09,2.77,50,71,44,33,21,35,65,41,74,9.56,4.93,2.00,3.40,3.1,1.20,1.53,2.33,4.40,1.60,2.39,2023-06-07,17:00:00,14,3,1,4,23,10,13,25,22/23,1
66077,İstanbulspor,Ümraniyespor,1.17,0.94,4,0,4,1,1,0,8,1,3,0,1,0,0,3,0,1,21,3,14,2,7,1,13,9,63,37,1.25,1.56,2.98,0.57,3.15,59,76,56,35,24,44,85,47,74,9.04,4.60,1.91,3.50,3.2,1.15,1.65,2.15,3.78,1.48,2.57,2023-06-07,17:00:00,9,4,0,4,24,16,8,22,22/23,1
66078,Ankaragücü,Adana Demirspor,1.22,1.61,1,2,3,1,0,1,7,1,0,0,0,0,0,0,0,0,13,9,8,6,5,3,13,9,51,49,1.65,1.82,1.96,1.40,3.00,68,88,59,32,27,35,71,62,88,10.30,4.35,2.30,3.50,2.5,1.10,1.73,1.95,3.15,1.39,3.11,2023-06-07,17:00:00,8,0,0,0,22,14,8,22,22/23,2


In [84]:
import pandas as pd

# Załóżmy, że 'data' to twój DataFrame
all_teams = pd.concat([data_clean['home_team_name'], data_clean['away_team_name']]).unique()
team_to_idx = {team: idx for idx, team in enumerate(all_teams)}

In [85]:
# Zamiana nazw drużyn na indeksy dla każdego meczu
data_clean['home_team_idx'] = data_clean['home_team_name'].map(team_to_idx)
data_clean['away_team_idx'] = data_clean['away_team_name'].map(team_to_idx)

C:\Users\Łukasz\AppData\Local\Temp\ipykernel_5028\1234282150.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['home_team_idx'] = data_clean['home_team_name'].map(team_to_idx)
C:\Users\Łukasz\AppData\Local\Temp\ipykernel_5028\1234282150.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_clean['away_team_idx'] = data_clean['away_team_name'].map(team_to_idx)


In [86]:
from sklearn.model_selection import train_test_split

# Lista kolumn do wykluczenia
exclude_columns = [
    'result', 'home_team_goal_count', 'away_team_goal_count', 'total_goal_count', 
    'home_team_name', 'away_team_name', 'date', 'time', 'season'
]

# Wybór kolumn, które są numeryczne i nie znajdują się na liście wykluczeń
features_columns = [col for col in data_clean.columns if data_clean[col].dtype in [np.float64, np.int64] and col not in exclude_columns]

# Przygotuj dane dla modelu
X = data_clean[features_columns]  # Dodaj inne cechy jako potrzebne
y = data_clean['result']  # Lub inna kolumna zależna od zadania

# Podział danych na zestawy treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
# Załóżmy, że 'team_to_idx' to słownik mapujący nazwy drużyn na unikalne indeksy
home_teams = torch.tensor([team_to_idx[name] for name in data_clean['home_team_name']])
away_teams = torch.tensor([team_to_idx[name] for name in data_clean['away_team_name']])
import pandas as pd
import torch

# Lista kolumn do wykluczenia
exclude_columns = [
    'result', 'home_team_goal_count', 'away_team_goal_count', 'total_goal_count', 
    'home_team_name', 'away_team_name', 'date', 'time', 'season'
]

# Wybór kolumn, które są numeryczne i nie znajdują się na liście wykluczeń
features_columns = [col for col in data_clean.columns if data_clean[col].dtype in [np.float64, np.int64] and col not in exclude_columns]

# Tworzenie tensora z wybranych cech
features = torch.tensor(data_clean[features_columns].values.astype(np.float32))


In [57]:
data_clean.to_csv('./data/model_data.csv')

In [61]:
data_loader

In [62]:
model = FootballModel(num_teams=len(team_to_idx), embedding_dim=50, num_features=features.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()  # Dla regresji liczby goli; użyj CrossEntropyLoss dla klasyfikacji wyniku

from torch.utils.data import DataLoader, TensorDataset

# Przygotowanie danych
dataset = TensorDataset(features, torch.tensor(data_clean['total_goal_count'].values, dtype=torch.float32))
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)  # Użyj mniejszego batch_size, jeśli jest taka potrzeba

# Pętla treningowa z użyciem DataLoader
# for epoch in range(100):
#     for batch in data_loader:
#         features_batch, targets_batch = batch
#         optimizer.zero_grad()
#         predictions = model(home_teams, away_teams, features_batch)
#         loss = criterion(predictions, targets_batch)
#         loss.backward()
#         optimizer.step()
#     print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# W pętli treningowej, upewnij się, że przekazujesz tylko indeksy dla bieżącej partii
for epoch in range(100):  # Liczba epok
    for batch in data_loader:
        features_batch, targets_batch = batch
        optimizer.zero_grad()

        # Aktualizacja: Pobieranie indeksów drużyn dla bieżącej partii
        home_teams_batch = home_teams[batch.indices]  # Dostosuj, jeśli masz inną metodę indeksowania
        away_teams_batch = away_teams[batch.indices]

        # Wykonanie predykcji
        predictions = model(home_teams_batch, away_teams_batch, features_batch)

        # Obliczanie straty i wykonanie kroku optymalizacyjnego
        loss = criterion(predictions, targets_batch)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


AttributeError: 'list' object has no attribute 'indices'

In [67]:
from torch.utils.data import DataLoader, TensorDataset

# Przygotowanie danych
dataset = TensorDataset(features, torch.tensor(data_clean['total_goal_count'].values, dtype=torch.float32))
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

# Pętla treningowa z manualnym śledzeniem indeksów
for epoch in range(100):
    for i, (features_batch, targets_batch) in enumerate(data_loader):
        batch_start = i * data_loader.batch_size
        batch_end = batch_start + features_batch.size(0)

        # Wyodrębnienie odpowiednich indeksów dla home_teams i away_teams
        home_teams_batch = home_teams[batch_start:batch_end]
        away_teams_batch = away_teams[batch_start:batch_end]

        optimizer.zero_grad()

        # Wykonanie predykcji
        predictions = model(home_teams_batch, away_teams_batch, features_batch)

        # Obliczanie straty i wykonanie kroku optymalizacyjnego
        loss = criterion(predictions, targets_batch)
        loss.backward()
        optimizer.step()

    if epoch % 5 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

c:\ProgramData\anaconda3\envs\magisterka_env\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


c:\ProgramData\anaconda3\envs\magisterka_env\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([57])) that is different to the input size (torch.Size([57, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Loss: 2.2620880603790283
Epoch 6, Loss: 3.7621076107025146
Epoch 11, Loss: 1.7562406063079834
Epoch 16, Loss: 2.621856927871704
Epoch 21, Loss: 2.876157760620117
Epoch 26, Loss: 2.858184814453125
Epoch 31, Loss: 2.6403555870056152
Epoch 36, Loss: 3.475630521774292
Epoch 41, Loss: 3.6825263500213623
Epoch 46, Loss: 3.255009412765503
Epoch 51, Loss: 2.8816211223602295
Epoch 56, Loss: 3.0204551219940186
Epoch 61, Loss: 2.4734113216400146
Epoch 66, Loss: 2.022355318069458
Epoch 71, Loss: 2.740805149078369
Epoch 76, Loss: 3.154503107070923
Epoch 81, Loss: 3.3119187355041504
Epoch 86, Loss: 3.5286076068878174
Epoch 91, Loss: 3.0966293811798096
Epoch 96, Loss: 2.0056114196777344


In [71]:
from sklearn.model_selection import train_test_split

In [ ]:
features_columns = [col for col in data_clean.columns if data_clean[col].dtype in [np.float64, np.int64] and col not in exclude_columns]

In [77]:
X = data_clean[features_columns]
y = data_clean['total_goal_count']

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

In [81]:
# Zakładam, że X_test, y_test są już przygotowane przez train_test_split i są w formie numpy array
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Stwórz TensorDataset
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Stwórz DataLoader
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [82]:
# Zakładając, że masz zdefiniowany DataLoader dla danych testowych
test_loss = 0
num_batches = 0

model.eval()  # Przełącz model w tryb ewaluacji
with torch.no_grad():  # Wyłączenie obliczeń gradientów
    for features_batch, targets_batch in test_loader:
        predictions = model(home_teams_batch, away_teams_batch, features_batch)
        loss = criterion(predictions, targets_batch)
        test_loss += loss.item()
        num_batches += 1

average_test_loss = test_loss / num_batches
print(f'Average Test Loss: {average_test_loss}')


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 57 but got size 64 for tensor number 2 in the list.